In [2]:
pip install selenium

  Using cached selenium-4.10.0-py3-none-any.whl (6.7 MB)
  Using cached trio-0.22.1-py3-none-any.whl (399 kB)
  Using cached trio_websocket-0.10.3-py3-none-any.whl (17 kB)
  Using cached outcome-1.2.0-py2.py3-none-any.whl (9.7 kB)
  Using cached exceptiongroup-1.1.2-py3-none-any.whl (14 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
Note: you may need to restart the kernel to use updated packages.


In [27]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
import os

# Chrome WebDriver 옵션 설정
options = Options()
# options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# Chrome WebDriver 실행
driver = webdriver.Chrome(options=options)

# 데이터를 저장할 리스트
data_list = []

# 숫자 2부터 1897까지 반복
for cookid in range(2, 5):
    # URL로 이동합니다.
    url = f"https://www.menupan.com/Cook/recipeview.asp?cookid={cookid}"
    driver.get(url)

    # 2초 대기
    time.sleep(2)

    # 데이터를 저장할 딕셔너리
    data = {}

    # 제목수집
    wrap_top = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "wrap_top")))
    recipe_title = wrap_top.find_element(By.TAG_NAME, "h2").text
    article_text = wrap_top.find_element(By.CLASS_NAME, "article").text
    data["Recipe Title"] = recipe_title
    data["Article Text"] = article_text

    # restName 수집
    wrap_info = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "wrap_info")))

    # dt와 dd 요소 찾기
    dt_elements = wrap_info.find_elements(By.TAG_NAME, "dt")
    dd_elements = wrap_info.find_elements(By.TAG_NAME, "dd")

    # dt와 dd 요소의 글자를 추출하여 data 딕셔너리에 추가
    for dt, dd in zip(dt_elements, dd_elements):
        dt_text = dt.text.strip()
        dd_text = dd.text.strip()
        data[dt_text] = dd_text

    # 재료정보 수집
    tab_info = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "wrap_food")))

    # dt와 dd 요소 찾기
    dt_elements = tab_info.find_elements(By.TAG_NAME, "dt")
    dd_elements = tab_info.find_elements(By.TAG_NAME, "dd")

    # dt와 dd 요소의 글자를 추출하여 data 딕셔너리에 추가
    for dt, dd in zip(dt_elements, dd_elements):
        dt_text = dt.text.strip()
        dd_text = dd.text.strip()
        data[dt_text] = dd_text

    # 요리법 수집
    wrap_recipe = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "wrap_recipe")))

    # dt와 dd 요소 찾기
    dt_elements = wrap_recipe.find_elements(By.TAG_NAME, "dt")
    dd_elements = wrap_recipe.find_elements(By.TAG_NAME, "dd")

    # dt와 dd 요소의 글자를 추출하여 data 딕셔너리에 추가
    for dt, dd in zip(dt_elements, dd_elements):
        dt_text = dt.text.strip()
        dd_text = dd.text.strip()
        data[dt_text] = dd_text

    # 데이터를 리스트에 추가
    data_list.append(data)

# WebDriver를 종료합니다.
driver.quit()

# 데이터를 DataFrame으로 변환합니다.
df = pd.DataFrame(data_list)

# Excel 파일 경로
file_path = "menupantext.xlsx"

# 데이터를 Excel 파일로 저장합니다.
df.to_excel(file_path, index=False)
